In [ ]:
#| default_exp db

# db -- Database interace
> Methods for interfacing with a sqlite db

In [ ]:
#| export
from fastcore.utils import *
from sqlmodel import Session, select, create_engine, SQLModel, Field, Relationship

In [ ]:
#| hide
from nbdev.showdoc import *
from fastcore.test import *

In [ ]:
#| export
class PersonBase(SQLModel):
    "Base model for `Person`"
    first_name: str # First name of the Person
    last_name: Optional[str] = '' # Last name of the Person
    occupation: Optional[str] = '' # Job of the Person
        
class Person(PersonBase, table=True):
    "Person model containing an `id`, doubling up as the database model"
    id: Optional[int] = Field(default=None, primary_key=True)
        
class PersonRead(PersonBase):
    "Version of Person for reading from the db"
    id: int

class PersonCreate(PersonBase):
    pass

class PersonUpdate(SQLModel):
    "Version of Person for updating in the db"
    first_name: Optional[str] = ''
    last_name: Optional[str] = ''
    occupation: Optional[str] = ''

In [ ]:
#| export
sqlite_file_name = 'database.db'
sqlite_url = f'sqlite:///{sqlite_file_name}'
engine = create_engine(sqlite_url)

def create_db_and_tables():
    "Function which first creates the tables from the meta data generated by defining the SQLModels above."
    SQLModel.metadata.create_all(engine)
    create_persons()

def create_persons():
    "Create `Person`s add add to the database just so that it isn't empty"
    with Session(engine) as session:
        person_ifan = Person(first_name="ifan", last_name="johnston", occupation="dev")
        person_ifan2 = Person(first_name="ifan2", last_name="johnston2", occupation="researcher")
        session.add(person_ifan)
        session.add(person_ifan2)
        session.commit()

In [ ]:
#| export
def create_person(
               person: PersonCreate # An SQLModel for creating a `Person`
              )-> PersonRead:
    "Add a person to the database"
    with Session(engine) as session:
        db_person = Person.from_orm(person)
        session.add(person)
        session.commit()
        session.refresh(db_person)
    return db_person

def read_persons(
                offset: int = 0, # Offset to read from.
                limit: int = 100 # Maximum index to read to.
                ) -> List[Person]:
    with Session(engine) as session:
        persons = session.exec(select(Person).offset(offset).limit(limit)).all()
    return persons

In [ ]:
create_db_and_tables()

In [ ]:
read_persons()

[Person(last_name='johnston', occupation='dev', first_name='ifan', id=1),
 Person(last_name='johnston', occupation='dev', first_name='ifan', id=2),
 Person(last_name='johnston2', occupation='researcher', first_name='ifan2', id=3)]